# Handcrafted Features

## Imports

In [ ]:

########################### (https://github.com/curiousily/TensorFlow-on-Android-for-Human-Activity-Recognition-with-LSTMs/blob/master/human_activity_recognition.ipynb) imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42

#!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from keras import backend as K
from sklearn.utils import shuffle

from matplotlib.pyplot import figure


########################### (https://github.com/curiousily/TensorFlow-on-Android-for-Human-Activity-Recognition-with-LSTMs/blob/master/human_activity_recognition.ipynb) imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle




# Upload Data 

---------------------------------



Train

In [ ]:
from google.colab import files
uploaded_signal_train = files.upload()

Test

In [ ]:
from google.colab import files
uploaded_signal_test = files.upload()

### Reading in segments & labels

Train

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_train = [key for key in uploaded_signal_train.keys()]


# set up list to hold all loaded npy 
all_dataframe_train = [] 


for i in range(len(list_of_dataframes_train)):

    # load in the data 
    dataframe_train = pd.read_csv(list_of_dataframes_train[i])


    # append the data to 'all' list
    all_dataframe_train.append(dataframe_train) 


all_df_train = pd.concat(all_dataframe_train)

Test

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_test = [key for key in uploaded_signal_test.keys()]


# set up list to hold all loaded npy 
all_dataframe_test = [] 


for i in range(len(list_of_dataframes_test)):

    # load in the data 
    dataframe_test = pd.read_csv(list_of_dataframes_test[i])


    # append the data to 'all' list
    all_dataframe_test.append(dataframe_test) 


all_df_test = pd.concat(all_dataframe_test)


# Quick Shuffle 

In [ ]:
all_df_train = shuffle(all_df_train)
all_df_test = shuffle(all_df_test)

# Quick Look 

In [ ]:
all_df_train.head()

In [ ]:
all_df_test.head()

# Train Test Split 

In [ ]:
# Getting X_train & y_train'
X_train = all_df_train.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run', 'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'	], axis = 1)
#X_train = all_df_train.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run'	], axis = 1)

y_train = all_df_train['Label_segment'].values


# convert to dummy category
y_train_dum = pd.get_dummies(y_train)

print(f'\nDummy labels Shape {y_train_dum.shape}')

In [ ]:
# Getting X_train & y_train

X_test = all_df_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run',  'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'], axis = 1)
#X_test = all_df_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run'	], axis = 1)

y_test = all_df_test['Label_segment'].values

# convert to dummy category
y_test_dum = pd.get_dummies(y_test)

print(f'\nDummy labels Shape {y_test_dum.shape}')

--------------------------------------

Standard Scale 

In [ ]:
# Set up ss for non-shuffled data
ss = StandardScaler()

# fit for non-shuffled
X_train_scale = ss.fit_transform(X_train)
X_test_scale = ss.transform(X_test)


---------------------------

Deep 

In [ ]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from matplotlib import pyplot

from keras.optimizers import Adam

import time
import tensorflow as tf

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_dum.shape[1]

# set up model
model = Sequential()

#input layer
model.add(Dense(100, activation='relu' , input_shape = (X_train_scale.shape[1], )))

# hidden layers
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))


# output layer
model.add(Dense(n_outputs, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
X_train_scale.shape

Fit 

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
history = model.fit(X_train_scale, y_train_dum,
                    validation_data=(X_test_scale, y_test_dum),
                    epochs = 100,
                    batch_size=200,
                    callbacks=[earlyStopping, mcp_save]
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

Learning Curves

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['categorical_accuracy']
val_accuracy = history_dict['val_categorical_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

------------------------

Reports

Make Predicitions

In [ ]:
# For Confusion matrix & Classification Report 

y_preds = model.predict(X_test_scale)
max_predictions = np.argmax(y_preds, axis=1)

max_test = np.argmax(np.asarray(y_test_dum), axis=1)

##################


y_preds_train = model.predict(X_train_scale)

max_predictions_train = np.argmax(y_preds_train, axis=1)
max_train = np.argmax(np.asarray(y_train_dum), axis=1)


# Metrics 

Test

In [ ]:
from sklearn.metrics import accuracy_score , precision_score
from sklearn.metrics import balanced_accuracy_score, recall_score
from sklearn.metrics import f1_score


print("Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
print("Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))
print("Test Precision Score : \t\t" , precision_score(max_test, max_predictions , average='weighted'))
print("Test Recall Score : \t\t" , recall_score(max_test, max_predictions , average='weighted'))

Train

In [ ]:
print("Train Accuracy : \t\t" ,accuracy_score(max_train, max_predictions_train))
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(max_train, max_predictions_train))
print("Train F1 Score : \t\t" , f1_score(max_train, max_predictions_train, average='weighted'))

Imports 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# set up labels 
LABELS = ['Go', 'Turn1',  'Turn2' , 'Walk1', 'Walk2', 'Sit']


# Test Set 

In [ ]:
print(classification_report(max_test, max_predictions , target_names=LABELS))


# Train Set 

In [ ]:
print(classification_report(max_train, max_predictions_train , target_names=LABELS))

--------------------

# Confusion Matrix 

In [ ]:
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions  )

confusion_matrix_t = metrics.confusion_matrix(max_train, max_predictions_train )

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS,  annot=True, fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

-------------------------------------

<br>

<br>

---------------------------------------

Check if SMOTE helps

In [ ]:
print(Counter(y_train))

In [ ]:

# transform the dataset
oversample = SMOTE()
X_train_smote, y_train_smote = oversample.fit_resample(X_train_scale, y_train)

# convert to dummy category
y_train_smote_dum = pd.get_dummies(y_train_smote)

print(f'\nDummy labels Shape {y_train_smote_dum.shape}')

In [ ]:
print(Counter(y_train_smote))

-----------------------------------------------------

-------------------------------

Deep + Smote

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_dum.shape[1]

# set up model
model_smote = Sequential()

#input layer
model_smote.add(Dense(50, activation='relu'))


# hidden layers
model_smote.add(Dense(50, activation='relu'))
model_smote.add(Dropout(0.5))
model_smote.add(Dense(50, activation='relu'))
#model_smote.add(Dropout(0.5))
#model_smote.add(Dense(100, activation='relu'))


# output layer
model_smote.add(Dense(n_outputs, activation='softmax'))

# compile model
model_smote.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
history_smote = model_smote.fit(X_train_smote, y_train_smote_dum,
                    validation_data=(X_test_scale, y_test_dum),
                    epochs = 100,
                    batch_size=200,
                    callbacks=[earlyStopping, mcp_save]
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

-----------------------------


Learning Curves

In [ ]:
history_dict = history_smote.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['categorical_accuracy']
val_accuracy = history_dict['val_categorical_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy, 'r', label='Training accuracy')
ax[0].plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax[0].set_title('Training & Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()
#
# Plot the loss vs Epochs
#
ax[1].plot(epochs, loss_values, 'r', label='Training loss')
ax[1].plot(epochs, val_loss_values, 'b', label='Validation loss')
ax[1].set_title('Training & Validation Loss', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Loss', fontsize=16)
ax[1].legend()

------------------------

Reports

Make Predicitions

In [ ]:
# For Confusion matrix & Classification Report 

y_preds = model_smote.predict(X_test_scale)
max_predictions = np.argmax(y_preds, axis=1)

max_test = np.argmax(np.asarray(y_test_dum), axis=1)

##################


y_preds_train = model_smote.predict(X_train_smote)

max_predictions_train = np.argmax(y_preds_train, axis=1)
max_train = np.argmax(np.asarray(y_train_smote_dum), axis=1)


# Metrics 

Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

print("Test Accuracy : \t\t" ,accuracy_score(max_test, max_predictions))
print("Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
print("Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))

Train

In [ ]:
print("Train Accuracy : \t\t" ,accuracy_score(max_train, max_predictions_train))
print("Train Balanced Accuracy : \t" , balanced_accuracy_score(max_train, max_predictions_train))
print("Train F1 Score : \t\t" , f1_score(max_train, max_predictions_train, average='weighted'))

Imports 

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

LABELS = ['Go', 'Walk1', 'Turn1', 'Walk2', 'Turn2', 'Sit']


# Test Set 

In [ ]:
print(classification_report(max_test, max_predictions , target_names=LABELS))


# Train Set 

In [ ]:
print(classification_report(max_train, max_predictions_train , target_names=LABELS))

--------------------

# Confusion Matrix 

In [ ]:
confusion_matrix = metrics.confusion_matrix(max_test, max_predictions  )

confusion_matrix_t = metrics.confusion_matrix(max_train, max_predictions_train )

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS,  annot=True, fmt="d");
plt.title("Test Data Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(confusion_matrix_t, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Train Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show();

-------------------------------------

# Transfer Learning Touch Up

In [ ]:
from google.colab import files
uploaded_dataframe_train_id = files.upload()

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_train_trans = [key for key in uploaded_dataframe_train_id.keys()]


# set up list to hold all loaded npy 
all_dataframe_train_trans = [] 


for i in range(len(list_of_dataframes_train_trans)):

    # load in the data 
    dataframe_train = pd.read_csv(list_of_dataframes_train_trans[i])


    # append the data to 'all' list
    all_dataframe_train_trans.append(dataframe_train) 


all_df_train_trans = pd.concat(all_dataframe_train_trans)

In [ ]:
from google.colab import files
uploaded_dataframe_validation_id = files.upload()

In [ ]:
# getting keys, which is file names of df
list_of_dataframes_val_trans = [key for key in uploaded_dataframe_validation_id.keys()]


# set up list to hold all loaded npy 
all_dataframe_val_trans = [] 


for i in range(len(list_of_dataframes_val_trans)):

    # load in the data 
    dataframe_val = pd.read_csv(list_of_dataframes_val_trans[i])


    # append the data to 'all' list
    all_dataframe_val_trans.append(dataframe_val) 


all_df_val_trans = pd.concat(all_dataframe_val_trans)

--------------------------

In [ ]:
# Getting X_train & y_train
X_train_trans = all_df_train_trans.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run',  'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'], axis = 1)
y_train_trans = all_df_train_trans['Label_segment'].values

# convert to dummy category
y_train_trans_dum = pd.get_dummies(y_train_trans)

# scale data using fitted ss
X_train_trans_scale = ss.transform(X_train_trans)


In [ ]:
# Getting X_val & y_val
X_val_trans = all_df_val_trans.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run',  'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'], axis = 1)
y_val_trans = all_df_val_trans['Label_segment'].values

# convert to dummy category
y_val_trans_dum = pd.get_dummies(y_val_trans)

# scale data using fitted ss
X_val_trans_scale = ss.transform(X_val_trans)


-------------------------

Now give quick train on specific case "Calibration"

In [ ]:
# Change learning rate from default adam = 0.001 , to 0.0001 and train for handful more epochs on specific participant data 
K.set_value(model.optimizer.learning_rate, 0.001)
#K.set_value(model_smote.optimizer.learning_rate, 0.001)

print("Learning rate before second fit:", model.optimizer.learning_rate.numpy())
#print("Learning rate before second fit:", model_smote.optimizer.learning_rate.numpy())

Fit Specfic ID

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
new_history = model.fit(X_train_trans_scale, y_train_trans_dum,
                    validation_data=(X_val_trans_scale, y_val_trans_dum),
                    epochs = 3,
                    batch_size= 200,
                    callbacks = [earlyStopping]
                    
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

**Smote Model**

In [ ]:
# just for timing model
t0 = time.time()

# fitting model 
new_history = model_smote.fit(X_train_trans_scale, y_train_trans_dum,
                    validation_data=(X_val_trans_scale, y_val_trans_dum),
                    epochs = 5,
                    batch_size= 200,
                    callbacks = [earlyStopping]
                    
                    )


t1 = time.time()

total = t1-t0

print(f'Time Taken: {total}')

---------------------------------------------

In [ ]:
def holdout_checker(smote = False):

  # Read in Fresh Data to see what predictions looks like
  from google.colab import files
  uploaded_test = files.upload()

  # getting keys, which is file names of csv
  val_file_name = [key for key in uploaded_test.keys()]

  # read in csv 
  signal_test = pd.read_csv(val_file_name[0])


  # Getting X_train & y_train
  X_data = signal_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run','X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands' ], axis = 1)
  #X_data = signal_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run' ], axis = 1)

  y_data = signal_test['Label_segment'].values

  # scale with already fit ss 
  X_data_scale = ss.transform(X_data)

  # convert to dummy category
  y_data_dum = pd.get_dummies(y_data)
  
  # Take max of dummy classifier
  y_data = np.argmax(np.asarray(y_data_dum), axis=1)

  # make prediction with model

  # if selecting smote trained model
  if smote == True:
    hold_preds = model_smote.predict(X_data_scale)

  # else non-smote trained model
  else:
    hold_preds = model.predict(X_data_scale)

  # take max of predictions 
  max_predictions = np.argmax(hold_preds, axis=1)

  # metrics 
  print("\n---------------------- Metrics ----------------------------------------")

  #print("Accuracy : \t\t" ,accuracy_score(y_data, max_predictions))
  print("Balanced Accuracy : \t" , balanced_accuracy_score(y_data, max_predictions))
  print("F1 Score : \t\t" , f1_score(y_data, max_predictions, average='weighted'))
  print("Precision Score : \t\t" , precision_score(y_data, max_predictions, average='weighted'))
  print("Recall Score : \t\t" , recall_score(y_data, max_predictions, average='weighted'))


  # set up labels 
  LABELS = ['Go', 'Turn1',  'Turn2' , 'Walk1', 'Walk2', 'Sit']

  # classification report 
  print("\n------------------- HoldOut Classification Report ---------------")
  print(classification_report(y_data ,max_predictions))
  print(" ")

  # confusion matrix
  confusion_matrix_out = metrics.confusion_matrix(y_data, max_predictions )

  plt.figure(figsize=(14, 10))
  sns.heatmap(confusion_matrix_out, xticklabels=LABELS, yticklabels=LABELS, annot=True ,fmt="d" )



  plt.title("HoldOut Data Confusion matrix")
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

  # lastly just printing actual predictions
  print("\n0 = Go \t\t 1 = Turn 1 \t 2 = Turn 2")
  print("\n3 = Walk 1 \t 4 = Walk 2 \t 5 = Sit")

  print(" ")
  print(max_predictions)

  print("\nTest Counts" ,Counter(max_predictions))
  print("\nReal Counts" ,Counter(y_data))



In [ ]:
holdout_checker()

In [ ]:
holdout_checker(smote=True)

In [ ]:
holdout_checker()

In [ ]:
holdout_checker(smote=True)

<br>

<br>

--------------------

Hyper Tuning 


In [ ]:
from keras.optimizers import Adam , RMSprop, Adagrad

In [ ]:
optimisers = ['Adam' , 'RMSprop', 'Adagrad']

batch_sizes = [50,200,1000]

#learning_rate = [0.001 , 0.01, 0.1]

learning_rate = [0.00001 , 0.0001]

In [ ]:
#n_timesteps, n_features, n_outputs = X_train.shape[1] , X_train.shape[2] , y_train_dum.shape[1]

n_outputs =  y_train_dum.shape[1]

# set up model
model = Sequential()

#input layer
model.add(Dense(100, activation='relu' , input_shape = (X_train_scale.shape[1], )))


# hidden layers
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

# output layer
model.add(Dense(n_outputs, activation='softmax'))

model.save_weights('basemodel.h5')

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

 	
# patient early stopping
#es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5 , min_delta=0.05)


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.base_model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
from sklearn.metrics import accuracy_score, recall_score , precision_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

Fit 

In [ ]:
list_of_f1           = []
list_of_recall       = []
list_of_precision    = []
list_of_balacc       = []
list_of_learningrate = []
list_of_optim        = []
list_of_batch        = []



for i in range(len(optimisers)):

  for size in batch_sizes:

    for learn in learning_rate:

      # resets the weights back to random
      model.load_weights('basemodel.h5')

      # compile model
      model.compile(loss='categorical_crossentropy', optimizer= optimisers[i] , metrics=[tf.keras.metrics.CategoricalAccuracy()])

      # set learning rate
      K.set_value(model.optimizer.learning_rate, learn)

      # just for timing model
      t0 = time.time()

      # fitting model 
      history = model.fit(X_train_scale, y_train_dum,
                          validation_data=(X_test_scale, y_test_dum),
                          epochs = 100,
                          batch_size=size,
                          callbacks=[earlyStopping, mcp_save]
                          )


      t1 = time.time()

      total = t1-t0

      print(f'Time Taken: {total}')

      # For Confusion matrix & Classification Report 

      y_preds = model.predict(X_test_scale)
      max_predictions = np.argmax(y_preds, axis=1)

      max_test = np.argmax(np.asarray(y_test_dum), axis=1)

      ##################


      y_preds_train = model.predict(X_train_scale)

      max_predictions_train = np.argmax(y_preds_train, axis=1)
      max_train = np.argmax(np.asarray(y_train_dum), axis=1)


      #############################

      list_of_f1.append(f1_score(max_test, max_predictions , average='weighted'))          
      list_of_recall.append(recall_score(max_test, max_predictions , average='weighted'))       
      list_of_precision.append(precision_score(max_test, max_predictions , average='weighted'))    
      list_of_balacc.append(balanced_accuracy_score(max_test, max_predictions))       
      list_of_learningrate.append(learn) 
      list_of_optim.append(optimisers[i])
      list_of_batch.append(size)


      print("-"*50)
      print("\n")
      #Metrics
      print(f"{optimisers[i]} - Learning Rate {learn} - Batch Size: {size}  -  Test Balanced Accuracy : \t" , balanced_accuracy_score(max_test, max_predictions))
      print(f"{optimisers[i]} - Learning Rate {learn} - Batch Size: {size}  -  Test F1 Score : \t\t" , f1_score(max_test, max_predictions , average='weighted'))
      print(f"{optimisers[i]} - Learning Rate {learn} - Batch Size: {size}  -  Test Precision Score : \t\t" , precision_score(max_test, max_predictions , average='weighted'))
      print(f"{optimisers[i]} - Learning Rate {learn} - Batch Size: {size}  -  Test Recall Score : \t\t" , recall_score(max_test, max_predictions , average='weighted'))

      print("\n")
      print("-"*50)


In [ ]:
new_df = pd.DataFrame()

new_df['Optimizer'] = list_of_optim
new_df['BatchSize'] = list_of_batch
new_df['LearningRate'] = list_of_learningrate
new_df['F1-Score'] = list_of_f1
new_df['Balanced Accuracy'] = list_of_balacc
new_df['Recall Score'] = list_of_recall
new_df['Precision Score'] = list_of_precision



In [ ]:
new_df[ (new_df['LearningRate'] == 0.001)  | (new_df['LearningRate'] == 0.01)   ]